In [1]:
## %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
import tensorflow as tf
import math
import datetime

In [2]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)
        
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f).astype(np.float)
#         img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    # X = np.column_stack(images)
    X = np.stack(images, axis=0)
    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [3]:
# Load the data
data_root_path = 'cifar10-hw/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [4]:
print(X_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000,)


In [5]:
# Group's helper function
def split(X, y, val_size):
    '''
    split the data into training and validation set
    '''
    indices = np.random.permutation(X.shape[0])
    test_num = int(val_size * X.shape[0])
    return X[indices[test_num:]], X[indices[:test_num]], y[indices[test_num:]], y[indices[:test_num]]

In [6]:
X_trn, X_val, y_trn, y_val = split(X_train, y_train, val_size=0.1)
print(X_trn.shape)
print(X_val.shape)
print(y_trn.shape)
print(y_val.shape)

(45000, 32, 32, 3)
(5000, 32, 32, 3)
(45000,)
(5000,)


In [7]:
def normalize(raw_images):
    images = np.empty_like(raw_images)
    np.copyto(images, raw_images)
    for i in range(images.shape[0]):
        old = images[i]
        new = (old - np.mean(old)) / np.std(old)
        images[i, :, :, :] = new
    return images

In [8]:
X_trn = normalize(X_trn)
X_val = normalize(X_val)
X_tst = normalize(X_test)

In [9]:
# GLobal value
H, W, T = 32, 32, 10 # height/width of images, number of classes of images
cnns = dict()

In [10]:
class CNN():
    def __init__(self, model_fn, trainer, global_step=None):
        
        if global_step is None:
            tf.reset_default_graph()
            global_step = tf.Variable(0, trainable=False)
        self.iter_cnt = 0
        self.X = tf.placeholder(tf.float32, [None, H, W, 3])
        self.Y = tf.placeholder(tf.int64, [None])
        self.is_training = tf.placeholder(tf.bool)
        with tf.name_scope('loss'):
            logit, loss = model_fn(self.X, self.Y, self.is_training)
            tf.summary.scalar('loss', loss)
        # https://stackoverflow.com/a/43285333
        extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(extra_update_ops):
            with tf.variable_scope('optimizer'):
                gradients = trainer.compute_gradients(loss)
                train_op = trainer.apply_gradients(gradients, global_step=global_step)
        for tup in gradients:
            the_string = str(tup[1])
            if 'conv1/kernel:0' in the_string:
                tf.summary.histogram('Gradient/conv1', tup[0])
            elif 'conv2/kernel:0' in the_string:
                tf.summary.histogram('Gradient/conv2', tup[0])
            elif 'last/kernel:0' in the_string:
                tf.summary.histogram('Gradient/last_fc', tup[0])
        # train_op = trainer.minimize(loss)
        # Accuracy
        correct = tf.equal(tf.argmax(logit, 1), self.Y)
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
            tf.summary.scalar('accuracy', accuracy)
        self.sess = tf.Session()
        merged = tf.summary.merge_all()
        self.train_writer = tf.summary.FileWriter('/Users/jingxixu/Desktop/train', self.sess.graph)
        self.test_writer = tf.summary.FileWriter('/Users/jingxixu/Desktop/test')
        self.variables = {
            'train': [merged, loss, correct, train_op],
            'validate': [merged, loss, correct, accuracy],
            'test': logit
        }
        self.sess.run(tf.global_variables_initializer())
        
    def batch_gen(self, Xd, Yd, batch_size, shuffle=True):
        indicies = np.arange(Xd.shape[0])
        if shuffle:
            np.random.shuffle(indicies)
        for i in range(int(math.ceil(Xd.shape[0] / batch_size))):
            start_idx = (i * batch_size) % Xd.shape[0]
            idx = indicies[start_idx:start_idx + batch_size]
            yield Xd[idx, :], Yd[idx]

    def run(self, Xd, Yd, epochs, batch_size, print_every, plot_losses, status):
        iter_cnt = 0
        for e in range(epochs):
            correct = 0
            losses = []
            for Xb, Yb in self.batch_gen(Xd, Yd, batch_size, shuffle = True):
                self.iter_cnt += 1
                feed_dict = {self.X: Xb, self.Y: Yb, self.is_training: status=='train'}
                summary, loss, corr, _ = self.sess.run(self.variables[status], feed_dict = feed_dict)
                if status == 'train':
                    self.train_writer.add_summary(summary, self.iter_cnt)
                elif status == 'validate':
                    self.test_writer.add_summary(summary, self.iter_cnt)
                losses.append(loss)
                correct += np.sum(corr)
                if status == 'train' and iter_cnt % print_every == 0:
                    print("{} Iter {}: batch trn loss = {:.3f}, accuracy = {:.3f}".format(
                        datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        iter_cnt,
                        loss,
                        np.mean(corr),
                    ))
                iter_cnt += 1
            epoch_loss = np.mean(losses)
            epoch_accuracy = correct / Xd.shape[0]
            print("Epoch {}: mean loss = {:.3f}, accuracy = {:.3f}".format(
                e, epoch_loss, epoch_accuracy))
            if plot_losses:
                plt.plot(losses)
                plt.grid(True)
                plt.title('Epoch {} Mean Loss'.format(e+1))
                plt.xlabel('minibatch number')
                plt.ylabel('minibatch mean loss')
                plt.show()
        
        return epoch_loss, epoch_accuracy

    def train(self, Xd, Yd, epochs=1, batch_size=50, print_every=100, plot_losses=False):
        return self.run(Xd, Yd, epochs, batch_size, print_every, plot_losses, status='train')
            
    def validate(self, Xd, Yd, epochs=1, batch_size=50, print_every=100, plot_losses=False):
        return self.run(Xd, Yd, epochs, batch_size, print_every, plot_losses, status='validate')
    
    def predict(self, Xd):
        feed_dict = {self.X: Xd, self.is_training: False}
        logit = self.sess.run(self.variables['test'], feed_dict = feed_dict)
        return logit

## [0103] normalize + dropout
- base (0101): add normalize and dropout = 0.5
- batch size still 50
- trn: , val: (05 epochs)
- trn: , val: (10 epochs)
- trn: , val: (15 epochs)
- trn: , val: (20 epochs)

In [16]:
def model_fn(layer_input, labels, is_training):
    reg_scale= 0.1
    regularizer = tf.contrib.layers.l2_regularizer(scale=reg_scale)
    # initializer = tf.contrib.layers.xavier_initializer()
    initializer = None
    h, w = H, W
    # conv - bn - max_pool
    F1 = 64
    P1, S1 = 3, 2 # pool_size, strides
    layer = tf.layers.conv2d(
        inputs=layer_input, filters=F1, kernel_size=[5, 5], padding='same', activation=tf.nn.relu,
        kernel_regularizer=regularizer, kernel_initializer=initializer, name='conv1')
    layer = tf.layers.batch_normalization(
        inputs=layer, training=is_training,
        beta_regularizer=regularizer, gamma_regularizer=regularizer)
    layer = tf.layers.max_pooling2d(inputs=layer, pool_size=P1, strides=S1)
    h = math.floor((h - P1 + S1) / S1)
    w = math.floor((w - P1 + S1) / S1)
    # conv - bn - max_pool
    F2 = 64
    P2, S2 = 3, 2 # pool_size, strides
    layer = tf.layers.conv2d(
        inputs=layer, filters=F2, kernel_size=[5, 5], padding='same', activation=tf.nn.relu,
        kernel_regularizer=regularizer, kernel_initializer=initializer, name='conv2')
    layer = tf.layers.batch_normalization(
        inputs=layer, training=is_training,
        beta_regularizer=regularizer, gamma_regularizer=regularizer)
    layer = tf.layers.max_pooling2d(inputs=layer, pool_size=P2, strides=S2)
    h = math.floor((h - P2 + S2) / S2)
    w = math.floor((w - P2 + S2) / S2)
    # dense1 - bn - dropout - fc - softmax
    flat_size = F2 * h * w
    print(flat_size)
    layer = tf.reshape(layer, [-1, flat_size])
    layer = tf.layers.dense(
        inputs=layer, units=1024, activation=tf.nn.relu,
        kernel_regularizer=regularizer, kernel_initializer=initializer)
    layer = tf.layers.batch_normalization(
        inputs=layer, training=is_training,
        beta_regularizer=regularizer, gamma_regularizer=regularizer)
    layer = tf.layers.dropout(inputs=layer, rate=0.5, training=is_training)
    # no activation here for logit, as it will be calculated in loss
    logit = tf.layers.dense(
        inputs=layer, units=T, activation=None,
        kernel_regularizer=regularizer, kernel_initializer=initializer, name='last')
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int64), depth=10)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logit)
    
    return logit, loss


trainer = tf.train.GradientDescentOptimizer(learning_rate=0.03)
cnn = CNN(model_fn, trainer)
cnns['0103'] = cnn

3136


In [17]:
for i in range(10):
    print('train: epoch %d' % i)
    cnn.train(X_trn, y_trn, epochs=1, batch_size=50, print_every=100, plot_losses=False)
    print('validation')
    cnn.validate(X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train: epoch 0
2017-10-30 19:38:53 Iter 0: batch trn loss = 3.714, accuracy = 0.060
2017-10-30 19:39:22 Iter 100: batch trn loss = 2.642, accuracy = 0.320
2017-10-30 19:39:53 Iter 200: batch trn loss = 1.951, accuracy = 0.380
2017-10-30 19:40:24 Iter 300: batch trn loss = 1.616, accuracy = 0.500
2017-10-30 19:40:55 Iter 400: batch trn loss = 1.143, accuracy = 0.520
2017-10-30 19:41:26 Iter 500: batch trn loss = 1.747, accuracy = 0.460
2017-10-30 19:41:56 Iter 600: batch trn loss = 1.292, accuracy = 0.540
2017-10-30 19:42:27 Iter 700: batch trn loss = 1.239, accuracy = 0.520
2017-10-30 19:42:58 Iter 800: batch trn loss = 1.406, accuracy = 0.600
Epoch 0: mean loss = 1.591, accuracy = 0.492
validation
Epoch 0: mean loss = 1.179, accuracy = 0.604
train: epoch 1
2017-10-30 19:44:07 Iter 0: batch trn loss = 1.550, accuracy = 0.500
2017-10-30 19:44:39 Iter 100: batch trn loss = 0.958, accuracy = 0.600
2017-10-30 19:45:10 Iter 200: batch trn loss = 1.020, accuracy = 0.620
2017-10-30 19:45:42 I

In [ ]:
for i in range(10, 20):
    print('train: epoch %d' % i)
    cnn.train(X_trn, y_trn, epochs=1, batch_size=50, print_every=100, plot_losses=False)
    print('validation')
    cnn.validate(X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train: epoch 10
2017-10-30 20:42:25 Iter 0: batch trn loss = 0.410, accuracy = 0.820
2017-10-30 20:42:54 Iter 100: batch trn loss = 0.350, accuracy = 0.840
2017-10-30 20:43:25 Iter 200: batch trn loss = 0.229, accuracy = 0.920
2017-10-30 20:43:56 Iter 300: batch trn loss = 0.551, accuracy = 0.800
2017-10-30 20:44:29 Iter 400: batch trn loss = 0.386, accuracy = 0.860
2017-10-30 20:45:01 Iter 500: batch trn loss = 0.528, accuracy = 0.820
2017-10-30 20:45:33 Iter 600: batch trn loss = 0.347, accuracy = 0.880
2017-10-30 20:46:05 Iter 700: batch trn loss = 0.423, accuracy = 0.800
2017-10-30 20:46:36 Iter 800: batch trn loss = 0.314, accuracy = 0.860
Epoch 0: mean loss = 0.456, accuracy = 0.840
validation
Epoch 0: mean loss = 0.768, accuracy = 0.759
train: epoch 11
2017-10-30 20:47:45 Iter 0: batch trn loss = 0.434, accuracy = 0.840
2017-10-30 20:48:16 Iter 100: batch trn loss = 0.428, accuracy = 0.820
2017-10-30 20:48:47 Iter 200: batch trn loss = 0.582, accuracy = 0.780
2017-10-30 20:49:18

In [18]:
Y_pred = cnn.predict(Xd=X_tst)
Y_pred = Y_pred.T
save_predictions('ans1-uni.npy', Y_pred)

loaded_y = np.load('ans1-uni.npy')
print(loaded_y.shape)

(10, 10000)


In [19]:
yp = cnn.predict(X_val)
z = np.argmax(yp, axis=1)
print(z.shape)

(5000,)


In [20]:
np.sum(z == y_val) / 5000

0.76580000000000004

In [15]:
# 5 epochs
for i in range(15)
print('train')
cnn.train(X_trn, y_trn, epochs=1, batch_size=50, print_every=100, plot_losses=False)
print('validation')
cnn.validate(X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train
2017-10-30 19:37:03 Iter 0: batch trn loss = 4.129, accuracy = 0.100
2017-10-30 19:37:32 Iter 100: batch trn loss = 1.976, accuracy = 0.380
2017-10-30 19:38:03 Iter 200: batch trn loss = 1.884, accuracy = 0.360


KeyboardInterrupt: 

In [ ]:
# 10 epochs
print('train')
cnn.train(X_trn, y_trn, epochs=1, batch_size=50, print_every=100, plot_losses=False)
print('validation')
cnn.validate(X_val, y_val, epochs=1, batch_size=y_val.shape[0])

In [ ]:
# 5 epochs
print('train')
cnn.train(X_trn, y_trn, epochs=1, batch_size=50, print_every=100, plot_losses=False)
print('validation')
cnn.validate(X_val, y_val, epochs=1, batch_size=y_val.shape[0])

In [28]:
# 15 epochs
print('train')
cnn.train(m_X_trn, y_trn, epochs=5, batch_size=50, print_every=100, plot_losses=False)
print('validation')
cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train
2017-10-29 14:54:04 Iter 0: batch trn loss = 0.442, accuracy = 0.840
2017-10-29 14:54:36 Iter 100: batch trn loss = 0.553, accuracy = 0.860
2017-10-29 14:55:08 Iter 200: batch trn loss = 0.305, accuracy = 0.920
2017-10-29 14:55:40 Iter 300: batch trn loss = 0.427, accuracy = 0.880
2017-10-29 14:56:12 Iter 400: batch trn loss = 0.415, accuracy = 0.880
2017-10-29 14:56:44 Iter 500: batch trn loss = 0.399, accuracy = 0.820
2017-10-29 14:57:16 Iter 600: batch trn loss = 0.434, accuracy = 0.880
2017-10-29 14:57:48 Iter 700: batch trn loss = 0.358, accuracy = 0.860
2017-10-29 14:58:20 Iter 800: batch trn loss = 0.412, accuracy = 0.860
Epoch 0: mean loss = 0.441, accuracy = 0.845
2017-10-29 14:58:52 Iter 900: batch trn loss = 0.365, accuracy = 0.880
2017-10-29 14:59:24 Iter 1000: batch trn loss = 0.640, accuracy = 0.800
2017-10-29 14:59:56 Iter 1100: batch trn loss = 0.383, accuracy = 0.900
2017-10-29 15:00:28 Iter 1200: batch trn loss = 0.453, accuracy = 0.840
2017-10-29 15:01:00 Iter 

(0.70572072, 0.79359999999999997)

In [29]:
# 20 epochs
print('train')
cnn.train(m_X_trn, y_trn, epochs=5, batch_size=50, print_every=100, plot_losses=False)
print('validation')
cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train
2017-10-29 15:53:00 Iter 0: batch trn loss = 0.115, accuracy = 0.980
2017-10-29 15:53:34 Iter 100: batch trn loss = 0.162, accuracy = 0.980
2017-10-29 15:54:08 Iter 200: batch trn loss = 0.306, accuracy = 0.880
2017-10-29 15:54:41 Iter 300: batch trn loss = 0.385, accuracy = 0.840
2017-10-29 15:55:15 Iter 400: batch trn loss = 0.320, accuracy = 0.860
2017-10-29 15:55:52 Iter 500: batch trn loss = 0.191, accuracy = 0.940
2017-10-29 15:56:27 Iter 600: batch trn loss = 0.373, accuracy = 0.920
2017-10-29 15:57:00 Iter 700: batch trn loss = 0.351, accuracy = 0.880
2017-10-29 15:57:33 Iter 800: batch trn loss = 0.512, accuracy = 0.860
Epoch 0: mean loss = 0.290, accuracy = 0.896
2017-10-29 15:58:06 Iter 900: batch trn loss = 0.363, accuracy = 0.820
2017-10-29 15:58:40 Iter 1000: batch trn loss = 0.248, accuracy = 0.840
2017-10-29 15:59:13 Iter 1100: batch trn loss = 0.346, accuracy = 0.860
2017-10-29 15:59:47 Iter 1200: batch trn loss = 0.129, accuracy = 0.940
2017-10-29 16:00:21 Iter 

(0.7545625, 0.80220000000000002)

In [30]:
# 25 epochs
print('train')
cnn.train(m_X_trn, y_trn, epochs=5, batch_size=50, print_every=100, plot_losses=False)
print('validation')
cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train
2017-10-29 16:18:47 Iter 0: batch trn loss = 0.268, accuracy = 0.920
2017-10-29 16:19:21 Iter 100: batch trn loss = 0.071, accuracy = 0.960
2017-10-29 16:19:54 Iter 200: batch trn loss = 0.121, accuracy = 0.940
2017-10-29 16:20:27 Iter 300: batch trn loss = 0.147, accuracy = 0.960
2017-10-29 16:21:00 Iter 400: batch trn loss = 0.162, accuracy = 0.940
2017-10-29 16:21:34 Iter 500: batch trn loss = 0.143, accuracy = 0.960
2017-10-29 16:22:08 Iter 600: batch trn loss = 0.077, accuracy = 0.980
2017-10-29 16:22:44 Iter 700: batch trn loss = 0.201, accuracy = 0.960
2017-10-29 16:23:19 Iter 800: batch trn loss = 0.207, accuracy = 0.900
Epoch 0: mean loss = 0.203, accuracy = 0.927
2017-10-29 16:23:52 Iter 900: batch trn loss = 0.252, accuracy = 0.940
2017-10-29 16:24:26 Iter 1000: batch trn loss = 0.190, accuracy = 0.940
2017-10-29 16:24:59 Iter 1100: batch trn loss = 0.207, accuracy = 0.940
2017-10-29 16:25:33 Iter 1200: batch trn loss = 0.176, accuracy = 0.900
2017-10-29 16:26:06 Iter 

(0.79273933, 0.79679999999999995)

In [31]:
# 30 epochs
print('train')
cnn.train(m_X_trn, y_trn, epochs=5, batch_size=50, print_every=100, plot_losses=False)
print('validation')
cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train
2017-10-29 17:17:07 Iter 0: batch trn loss = 0.086, accuracy = 0.980
2017-10-29 17:17:40 Iter 100: batch trn loss = 0.197, accuracy = 0.940
2017-10-29 17:18:13 Iter 200: batch trn loss = 0.107, accuracy = 0.980
2017-10-29 17:18:45 Iter 300: batch trn loss = 0.257, accuracy = 0.860
2017-10-29 17:19:18 Iter 400: batch trn loss = 0.142, accuracy = 0.960
2017-10-29 17:19:50 Iter 500: batch trn loss = 0.284, accuracy = 0.900
2017-10-29 17:20:24 Iter 600: batch trn loss = 0.243, accuracy = 0.920
2017-10-29 17:20:57 Iter 700: batch trn loss = 0.089, accuracy = 0.980
2017-10-29 17:21:31 Iter 800: batch trn loss = 0.206, accuracy = 0.920
Epoch 0: mean loss = 0.147, accuracy = 0.947
2017-10-29 17:22:04 Iter 900: batch trn loss = 0.014, accuracy = 1.000
2017-10-29 17:22:37 Iter 1000: batch trn loss = 0.042, accuracy = 1.000
2017-10-29 17:23:10 Iter 1100: batch trn loss = 0.062, accuracy = 0.980
2017-10-29 17:23:43 Iter 1200: batch trn loss = 0.112, accuracy = 0.960
2017-10-29 17:24:16 Iter 

(0.86955673, 0.80159999999999998)

In [32]:
# 35 epochs
print('train')
cnn.train(m_X_trn, y_trn, epochs=5, batch_size=50, print_every=100, plot_losses=False)
print('validation')
cnn.validate(m_X_val, y_val, epochs=1, batch_size=y_val.shape[0])

train
2017-10-29 18:43:59 Iter 0: batch trn loss = 0.073, accuracy = 0.980
2017-10-29 18:44:31 Iter 100: batch trn loss = 0.135, accuracy = 0.960
2017-10-29 18:45:03 Iter 200: batch trn loss = 0.019, accuracy = 1.000
2017-10-29 18:45:36 Iter 300: batch trn loss = 0.039, accuracy = 0.980
2017-10-29 18:46:08 Iter 400: batch trn loss = 0.072, accuracy = 0.980
2017-10-29 18:46:40 Iter 500: batch trn loss = 0.268, accuracy = 0.900
2017-10-29 18:47:12 Iter 600: batch trn loss = 0.011, accuracy = 1.000
2017-10-29 18:47:44 Iter 700: batch trn loss = 0.275, accuracy = 0.820
2017-10-29 18:48:17 Iter 800: batch trn loss = 0.061, accuracy = 0.980
Epoch 0: mean loss = 0.105, accuracy = 0.962
2017-10-29 18:48:49 Iter 900: batch trn loss = 0.059, accuracy = 0.980
2017-10-29 18:49:21 Iter 1000: batch trn loss = 0.180, accuracy = 0.900
2017-10-29 18:49:53 Iter 1100: batch trn loss = 0.089, accuracy = 0.960
2017-10-29 18:50:25 Iter 1200: batch trn loss = 0.121, accuracy = 0.940
2017-10-29 18:50:57 Iter 

(0.88099295, 0.7994)